In [1]:
# variables = ['datadate', 'ticker', 'agr', 'bm', 'mom12m', 'mve', 'operprof', 'roeq', 'absacc', 'acc', 'aeavol', 'age', 'baspread', 'BETA', 'bm_ia', 
#              'cash', 'cashdebt', 'cashpr', 'cfp', 'cfp_ia', 'chatoia', 'chcsho', 'chempia', 'chfeps', 'chinv', 'chmom', 'chnanalyst', 
#              'chpmia', 'chtx', 'cinvest', 'convind', 'currat', 'depr', 'disp', 'divi', 'divo', 'dy', 'ear', 'egr', 'ep', 'fgr5yr', 
#              'gma', 'grcapx', 'grltnoa', 'herf', 'hire', 'idiovol', 'ill', 'indmom', 'invest', 'IPO', 'lev', 'mom1m', 'mom36m', 'ms', 
#              'mve_ia', 'nanalyst', 'nincr', 'orgcap', 'pchcapx_ia', 'pchcurrat', 'pchdepr', 'pchgm_pchsale', 'pchsale_pchinvt', 
#              'pchsale_pchrect', 'pchsale_pchxsga', 'pchsaleinv', 'pctacc', 'pricedelay', 'ps', 'rd', 'rd_mve', 'rd_sale', 'realestate', 
#              'retvol', 'roaq', 'roavol', 'roic', 'rsup', 'salecash', 'saleinv', 'salerec', 'secured', 'securedind', 'sfe', 'sgr', 'sin', 
#              'sp', 'std_dolvol', 'std_turn', 'stdcf', 'sue', 'tang', 'tb', 'turn', 'zerotrade']
variables = ['datadate', 'ticker', 'bm', 'mom12m', 'mve', 'ret_fwd_1']

In [27]:
import pandas as pd
import numpy as np

green_cleaned = pd.read_csv('../green cleaned.csv')
green_cleaned['ret_fwd_1'] = (green_cleaned.groupby('permno')['ret_excess'].shift(-1) )

/var/folders/w3/6myr5tjs6ls3zn1g3ywbc29r0000gn/T/ipykernel_8294/2012901010.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  green_cleaned = pd.read_csv('../green cleaned.csv')


In [29]:
green_cleaned = green_cleaned[green_cleaned['datadate'] < '2020-01-31']
green_cleaned.loc[green_cleaned['datadate'] == '2019-12-31', 'ret_fwd_1'] = np.nan

In [32]:
green_cleaned.to_csv('green_cleaned_pre_2020.csv', index=False)

In [2]:
#from agentics import AG

In [3]:
#green_cleaned = AG.from_csv('../green cleaned.csv', max_rows = 10)

In [4]:
#green_cleaned[0]

In [5]:
# for x in green_cleaned:
#     print(x.bm)

In [6]:
# #green_cleaned = green_cleaned.add_attribute('recommendation', description='Buy, hold, or sell recommendation for given ticker')
# from pydantic import BaseModel

# class Recommendation(BaseModel):
#     recommendation: str | None = None

# target = AG(atype = Recommendation, instructions='Determine a buy, hold, or sell for given ticker.')

In [7]:
# output = await (target << green_cleaned)
# output.pretty_print()

In [8]:
#green_cleaned = await green_cleaned.self_transduction(['bm', 'mve', 'mom12m', 'ticker'], ['recommendation'])

In [2]:
from dotenv import find_dotenv, load_dotenv
from getpass import getpass
import os

#load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

Enter your GEMINI_API_KEY: ········


In [4]:
import sqlite3
import pandas as pd
from crewai.tools import tool
from crewai import Agent, Task, Crew, Process
from agentics import AG
import time
from functools import wraps
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

# ============================================
# Rate Limiting Setup
# ============================================
class RateLimiter:
    def __init__(self, calls_per_minute=5):
        self.calls_per_minute = calls_per_minute
        self.interval = 60.0 / calls_per_minute
        self.last_call = 0
        self.call_count = 0
        
    def wait_if_needed(self):
        """Wait if necessary to respect rate limit."""
        current_time = time.time()
        time_since_last = current_time - self.last_call
        
        if time_since_last < self.interval:
            sleep_time = self.interval - time_since_last
            print(f"[Rate Limit] Waiting {sleep_time:.2f}s (Call #{self.call_count + 1})")
            time.sleep(sleep_time)
        
        self.last_call = time.time()
        self.call_count += 1

rate_limiter = RateLimiter(calls_per_minute=5)

def rate_limit(func):
    """Decorator with retry logic for rate limiting."""
    @wraps(func)
    @retry(
        retry=retry_if_exception_type((Exception,)),
        wait=wait_exponential(multiplier=1, min=4, max=60),
        stop=stop_after_attempt(5)
    )
    def wrapper(*args, **kwargs):
        rate_limiter.wait_if_needed()
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"[Error] {func.__name__} failed: {str(e)}")
            raise
    return wrapper

# ============================================
# Database Setup
# ============================================
def setup_firm_database(csv_path, db_path='firm_data.db', selected_columns=None):
    """
    Load your CSV into SQLite database (run this once).
    
    IMPORTANT: CSV must include a 'ret' column for forward returns
    (i.e., the return in the NEXT period after the characteristics)
    """
    print(f"Loading CSV from {csv_path}...")
    
    sample_df = pd.read_csv(csv_path, nrows=5)
    
    if selected_columns is None:
        print(f"\nAvailable columns ({len(sample_df.columns)} total):")
        for i, col in enumerate(sample_df.columns, 1):
            print(f"  {i}. {col}")
        
        response = input("\nPress Enter to use ALL columns, or type 'stop' to exit: ").strip()
        if response.lower() == 'stop':
            print("Exiting.")
            return
        selected_columns = sample_df.columns.tolist()
    else:
        missing = [col for col in selected_columns if col not in sample_df.columns]
        if missing:
            print(f"\n❌ Error: These columns don't exist: {missing}")
            return
    
    # Ensure 'ret_fwd_1' column is included for return analysis
    if 'ret_fwd_1' not in selected_columns:
        print("⚠ Warning: 'ret_fwd_1' column not found. Adding it for return analysis.")
        if 'ret_fwd_1' in sample_df.columns:
            selected_columns.append('ret_fwd_1')
    
    clean_columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_')
                    .replace('(', '').replace(')', '').replace('/', '_') 
                    for col in selected_columns]
    
    import os
    if os.path.exists(db_path):
        os.remove(db_path)
    
    chunk_size = 50000
    conn = sqlite3.connect(db_path)
    
    first_chunk = True
    chunk_num = 0
    total_rows = 0
    
    print("\nLoading data...")
    for chunk in pd.read_csv(csv_path, chunksize=chunk_size, usecols=selected_columns):
        chunk.columns = clean_columns
        chunk_num += 1
        total_rows += len(chunk)
        
        if first_chunk:
            chunk.to_sql('firms', conn, if_exists='replace', index=False)
            first_chunk = False
        else:
            chunk.to_sql('firms', conn, if_exists='append', index=False)
        
        print(f"  Chunk {chunk_num}: {total_rows:,} rows")
    
    cursor = conn.cursor()
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_ticker ON firms(ticker)")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_date ON firms(datadate)")
    conn.commit()
    conn.close()
    
    print(f"\n✓ Database created: {db_path} ({total_rows:,} rows)")
    return clean_columns

# ============================================
# Database Query Tools - WITH HISTORICAL RETURNS
# ============================================
@tool("query_firm_database")
@rate_limit
def query_firm_database(sql_query: str) -> str:
    """
    Execute SQL queries on the S&P 500 firm characteristics database.
    
    Available table: 'firms' (contains firm characteristics AND forward returns)
    
    IMPORTANT DATA NOTES:
    - 'mve' = log(market value of equity), i.e., log size
    - 'bm' = log(book to market)
    - 'mom12m' = 12-month momentum
    - 'ret_fwd_1' = forward return (return in NEXT period after characteristics measured)
    - ALL features (except ret_fwd_1) are standardized: mean = 0, standard deviation = 1
    - Date range: Jan 1990 - Dec 2019
    
    Example queries:
    - SELECT * FROM firms WHERE datadate < '2019-12-31' LIMIT 10
    - SELECT AVG(ret) FROM firms WHERE bm < -1 AND datadate < '2019-12-31'
    - SELECT ticker, bm, mom12m, ret FROM firms WHERE datadate BETWEEN '2015-01-01' AND '2019-11-30'
    
    Args:
        sql_query: A valid SQL SELECT query
    
    Returns:
        Query results as a formatted string
    """
    try:
        conn = sqlite3.connect('firm_data.db')
        df = pd.read_sql_query(sql_query, conn)
        conn.close()
        
        if df.empty:
            return "Query returned no results."
        
        result = f"Query returned {len(df)} rows:\n\n"
        result += df.to_string(index=False)
        
        return result
    except Exception as e:
        return f"Error executing query: {str(e)}"


@tool("explore_threshold_sensitivity")
@rate_limit
def explore_threshold_sensitivity(base_condition: str, char_to_vary: str, 
                                  threshold_range: str, start_date: str = '1990-01-01', 
                                  end_date: str = '2019-11-30') -> str:
    """
    Test how returns vary as you change a specific threshold.
    
    This helps find optimal thresholds by showing performance across a range.
    
    Args:
        base_condition: The base SQL condition template with PLACEHOLDER for the threshold
                       Example: "bm < PLACEHOLDER AND mom12m > 0.5"
        char_to_vary: The characteristic being varied (e.g., 'bm')
        threshold_range: Comma-separated thresholds to test (e.g., "-1.5,-1.2,-1.0,-0.8,-0.5")
        start_date: Start of analysis
        end_date: End of analysis
    
    Returns:
        Performance across different threshold values
    
    Example usage:
    explore_threshold_sensitivity(
        base_condition="bm < PLACEHOLDER AND mom12m > 0.5",
        char_to_vary="bm",
        threshold_range="-1.5,-1.2,-1.0,-0.8,-0.5"
    )
    """
    try:
        conn = sqlite3.connect('firm_data.db')
        
        thresholds = [float(t.strip()) for t in threshold_range.split(',')]
        
        results = []
        for threshold in thresholds:
            condition = base_condition.replace('PLACEHOLDER', str(threshold))
            
            query = f"""
            SELECT 
                COUNT(*) as n_obs,
                AVG(ret_fwd_1) as avg_return,
                STDEV(ret_fwd_1) as std_return
            FROM firms
            WHERE datadate >= '{start_date}' AND datadate <= '{end_date}'
            AND ({condition})
            AND mve IS NOT NULL AND bm IS NOT NULL AND mom12m IS NOT NULL AND ret_fwd_1 IS NOT NULL
            """
            
            df = pd.read_sql_query(query, conn)
            if not df.empty and df['n_obs'].values[0] > 0:
                results.append({
                    'threshold': threshold,
                    'n_obs': int(df['n_obs'].values[0]),
                    'avg_return': df['avg_return'].values[0],
                    'std_return': df['std_return'].values[0] if df['std_return'].values[0] else 0
                })
        
        conn.close()
        
        if not results:
            return "No results found for threshold exploration"
        
        result = f"""Threshold Sensitivity Analysis ({start_date} to {end_date}):

Base Condition: {base_condition}
Varying: {char_to_vary}

"""
        for r in results:
            sharpe = r['avg_return'] / r['std_return'] if r['std_return'] > 0 else 0
            result += f"\nThreshold = {r['threshold']:>6.2f}:"
            result += f"\n  Observations: {r['n_obs']:>8,}"
            result += f"\n  Avg Return:   {r['avg_return']*100:>7.3f}%"
            result += f"\n  Sharpe:       {sharpe:>7.3f}\n"
        
        # Find best threshold
        best = max(results, key=lambda x: x['avg_return'])
        result += f"\nBest threshold: {best['threshold']:.2f} with {best['avg_return']*100:.3f}% return"
        
        return result
    except Exception as e:
        return f"Error in threshold sensitivity: {str(e)}"

@tool("get_database_schema")
@rate_limit
def get_database_schema() -> str:
    """
    Get the schema of the firms database including all column names and types.
    
    CRITICAL DATA INFORMATION:
    - 'mve' = log(market value of equity), represents log firm size
    - 'bm' = log(book to market) - value factor
    - 'mom12m' = 12-month momentum
    - 'ret_fwd_1' = forward return (return AFTER characteristics measured)
    - ALL features (except ret_fwd_1) are standardized to mean=0, std=1
    - Values represent standard deviations from mean (z-scores)
    """
    try:
        conn = sqlite3.connect('firm_data.db')
        cursor = conn.cursor()
        
        cursor.execute("PRAGMA table_info(firms)")
        columns = cursor.fetchall()
        
        cursor.execute("SELECT COUNT(*) FROM firms")
        row_count = cursor.fetchone()[0]
        
        cursor.execute("SELECT MIN(datadate), MAX(datadate) FROM firms")
        date_range = cursor.fetchone()
        
        conn.close()
        
        schema = f"""Database: firm_data.db
Table: firms
Total rows: {row_count:,}
Date range: {date_range[0]} to {date_range[1]}

DATA STANDARDIZATION:
- 'mve' = log(market value of equity) - log firm size (STANDARDIZED z-score)
- 'bm' = log(book to market) - value factor (STANDARDIZED z-score)
- 'mom12m' = 12-month momentum (STANDARDIZED z-score)
- 'ret_fwd_1' = forward return - NOT standardized, raw return in decimal (e.g., 0.05 = 5% return)
- Firm characteristics are z-scores (mean=0, std=1)
- Returns are raw decimal returns

Columns ({len(columns)} total):
"""
        for col in columns:
            schema += f"  - {col[1]} ({col[2]})\n"
        
        return schema
    except Exception as e:
        return f"Error retrieving schema: {str(e)}"

@tool("analyze_characteristic_returns")
@rate_limit
def analyze_characteristic_returns(characteristic: str, start_date: str = '1990-01-01', end_date: str = '2019-11-30') -> str:
    """
    Analyze the relationship between a characteristic and future returns.
    
    This tool divides firms into portfolios based on characteristic values and
    shows average returns for each portfolio across the historical period.
    
    Args:
        characteristic: Column name ('bm', 'mom12m', 'mve')
        start_date: Start of analysis period (default: 1990-01-01)
        end_date: End of analysis period (default: 2019-11-30, BEFORE prediction date)
    
    Returns:
        Average returns by characteristic quintiles (portfolios sorted by characteristic)
    """
    try:
        conn = sqlite3.connect('firm_data.db')
        
        # Get data for the period
        query = f"""
        SELECT {characteristic}, ret_fwd_1
        FROM firms
        WHERE datadate >= '{start_date}' AND datadate <= '{end_date}'
        AND {characteristic} IS NOT NULL AND ret_fwd_1 IS NOT NULL
        """
        
        df = pd.read_sql_query(query, conn)
        conn.close()
        
        if df.empty:
            return f"No data found for {characteristic} between {start_date} and {end_date}"
        
        # Create quintile portfolios
        df['quintile'] = pd.qcut(df[characteristic], q=5, labels=['Q1_Low', 'Q2', 'Q3', 'Q4', 'Q5_High'])
        
        # Calculate average returns by quintile
        quintile_returns = df.groupby('quintile')['ret_fwd_1'].agg(['mean', 'median', 'count'])
        quintile_returns.columns = ['avg_return', 'median_return', 'n_obs']
        
        # Calculate characteristic ranges for each quintile
        char_ranges = df.groupby('quintile')[characteristic].agg(['min', 'max'])
        
        result = f"""Historical Return Analysis for {characteristic} ({start_date} to {end_date}):

Portfolio Performance (sorted by {characteristic}):
"""
        for idx in quintile_returns.index:
            result += f"\n{idx}: {characteristic} range [{char_ranges.loc[idx, 'min']:.2f}, {char_ranges.loc[idx, 'max']:.2f}]"
            result += f"\n  Avg Return: {quintile_returns.loc[idx, 'avg_return']*100:.2f}%"
            result += f"\n  Median Return: {quintile_returns.loc[idx, 'median_return']*100:.2f}%"
            result += f"\n  Observations: {int(quintile_returns.loc[idx, 'n_obs']):,}\n"
        
        # Calculate spread (Q5 - Q1)
        spread = (quintile_returns.loc['Q5_High', 'avg_return'] - quintile_returns.loc['Q1_Low', 'avg_return']) * 100
        result += f"\nSpread (Q5_High - Q1_Low): {spread:.2f}%"
        
        return result
    except Exception as e:
        return f"Error analyzing returns: {str(e)}"

@tool("analyze_combined_characteristics")
@rate_limit
def analyze_combined_characteristics(sql_condition: str, start_date: str = '1990-01-01', end_date: str = '2019-11-30') -> str:
    """
    Analyze returns for firms meeting ANY arbitrary SQL condition.
    
    This is the most flexible tool - you can test ANY combination of characteristics.
    
    Examples:
    - "bm < -1.0 AND mom12m > 0.5" (value + momentum)
    - "bm > 1.5 OR mom12m < -1.0" (expensive OR poor momentum)
    - "(bm < -0.8 AND mve > 0.5) OR mom12m > 1.5" (large value stocks OR strong momentum)
    - "bm < -1.0 AND mom12m > 0.5 AND mve > 0" (value + momentum + large cap)
    - "NOT (bm > 1.0 OR mom12m < -0.5)" (exclude expensive or negative momentum)
    
    Args:
        sql_condition: Any valid SQL WHERE clause using mve, bm, mom12m
        start_date: Start of analysis period
        end_date: End of analysis period (BEFORE prediction date)
    
    Returns:
        Average returns for firms meeting the condition vs. all others
    """
    try:
        conn = sqlite3.connect('firm_data.db')
        
        query = f"""
        SELECT 
            CASE 
                WHEN {sql_condition} THEN 'Target_Group'
                ELSE 'Other'
            END as group_type,
            COUNT(*) as n_obs,
            AVG(ret_fwd_1) as avg_return,
            AVG(ret_fwd_1) * 100 as avg_return_pct,
            STDEV(ret_fwd_1) as std_return,
            MIN(ret_fwd_1) as min_return,
            MAX(ret_fwd_1) as max_return
        FROM firms
        WHERE datadate >= '{start_date}' AND datadate <= '{end_date}'
        AND mve IS NOT NULL AND bm IS NOT NULL AND mom12m IS NOT NULL AND ret_fwd_1 IS NOT NULL
        GROUP BY group_type
        """
        
        df = pd.read_sql_query(query, conn)
        conn.close()
        
        if df.empty:
            return f"No data found for condition: {sql_condition}"
        
        result = f"""Combined Characteristic Analysis ({start_date} to {end_date}):

Condition: {sql_condition}

"""
        for _, row in df.iterrows():
            result += f"{row['group_type']}:\n"
            result += f"  Observations: {int(row['n_obs']):,}\n"
            result += f"  Avg Return: {row['avg_return']*100:.3f}%\n"
            if row['std_return'] is not None:
                sharpe = row['avg_return'] / row['std_return'] if row['std_return'] > 0 else 0
                result += f"  Std Dev: {row['std_return']*100:.3f}%\n"
                result += f"  Sharpe (approx): {sharpe:.3f}\n"
            result += f"  Min Return: {row['min_return']*100:.2f}%\n"
            result += f"  Max Return: {row['max_return']*100:.2f}%\n\n"
        
        if len(df) == 2:
            target = df[df['group_type'] == 'Target_Group']['avg_return'].values[0]
            other = df[df['group_type'] == 'Other']['avg_return'].values[0]
            spread = (target - other) * 100
            result += f"Spread (Target - Other): {spread:.3f}%\n"
            
            # Statistical significance (t-test approximation)
            target_n = df[df['group_type'] == 'Target_Group']['n_obs'].values[0]
            other_n = df[df['group_type'] == 'Other']['n_obs'].values[0]
            result += f"\nSignal strength: {spread:.3f}% spread with {int(target_n):,} target observations"
        
        return result
    except Exception as e:
        return f"Error in combined analysis: {str(e)}\nMake sure your SQL condition is valid."

@tool("test_strategy_historically")
@rate_limit
def test_strategy_historically(buy_condition: str, sell_condition: str,
                                start_date: str = '1990-01-01', end_date: str = '2019-11-30') -> str:
    """
    Test a proposed strategy on historical data to see if it would have worked.
    
    This allows you to test ANY buy and sell conditions with complete flexibility.
    
    Example buy_conditions:
    - "bm < -1.0 AND mom12m > 0.5" (value + momentum)
    - "(bm < -0.8 AND mve > 0.5) OR mom12m > 1.5" (large value OR strong momentum)
    - "bm < -1.2 AND mom12m > 0.7 AND mve > 0" (value + momentum + large cap)
    - "mom12m > 1.0 OR (bm < -1.5 AND mve > 0)" (momentum OR extreme value in large caps)
    
    Example sell_conditions:
    - "bm > 1.5 OR mom12m < -1.0" (expensive OR negative momentum)
    - "bm > 1.5 AND mom12m < -0.5" (expensive AND weak momentum)
    - "(bm > 1.0 AND mom12m < 0) OR mve < -1.5" (expensive with no momentum OR very small)
    
    Args:
        buy_condition: SQL WHERE clause for BUY signals
        sell_condition: SQL WHERE clause for SELL signals
        start_date: Start of backtest period
        end_date: End of backtest period (BEFORE prediction date)
    
    Returns:
        Historical performance of the strategy (avg returns for BUY vs SELL vs HOLD)
    """
    try:
        conn = sqlite3.connect('firm_data.db')
        
        query = f"""
        SELECT 
            CASE 
                WHEN {buy_condition} THEN 'BUY'
                WHEN {sell_condition} THEN 'SELL'
                ELSE 'HOLD'
            END as signal,
            COUNT(*) as n_obs,
            AVG(ret_fwd_1) as avg_return,
            STDEV(ret_fwd_1) as std_return,
            MIN(ret_fwd_1) as min_return,
            MAX(ret_fwd_1) as max_return
        FROM firms
        WHERE datadate >= '{start_date}' AND datadate <= '{end_date}'
        AND mve IS NOT NULL AND bm IS NOT NULL AND mom12m IS NOT NULL AND ret_fwd_1 IS NOT NULL
        GROUP BY signal
        """
        
        df = pd.read_sql_query(query, conn)
        conn.close()
        
        if df.empty:
            return "No data found for strategy test"
        
        result = f"""Historical Strategy Backtest ({start_date} to {end_date}):

BUY Condition: {buy_condition}
SELL Condition: {sell_condition}

Results:
"""
        for _, row in df.iterrows():
            result += f"\n{row['signal']} Signal:"
            result += f"\n  Count: {int(row['n_obs']):,} observations"
            result += f"\n  Avg Return: {row['avg_return']*100:.3f}%"
            if row['std_return'] is not None and row['std_return'] > 0:
                sharpe = row['avg_return'] / row['std_return']
                result += f"\n  Std Dev: {row['std_return']*100:.3f}%"
                result += f"\n  Sharpe (approx): {sharpe:.3f}"
            result += f"\n  Min Return: {row['min_return']*100:.2f}%"
            result += f"\n  Max Return: {row['max_return']*100:.2f}%"
            result += "\n"
        
        # Calculate spreads
        if len(df) >= 2:
            result += "\nPerformance Spreads:"
            buy_ret = df[df['signal'] == 'BUY']['avg_return'].values
            sell_ret = df[df['signal'] == 'SELL']['avg_return'].values
            hold_ret = df[df['signal'] == 'HOLD']['avg_return'].values
            
            if len(buy_ret) > 0 and len(sell_ret) > 0:
                spread_buy_sell = (buy_ret[0] - sell_ret[0]) * 100
                result += f"\n  BUY vs SELL: {spread_buy_sell:.3f}%"
            
            if len(buy_ret) > 0 and len(hold_ret) > 0:
                spread_buy_hold = (buy_ret[0] - hold_ret[0]) * 100
                result += f"\n  BUY vs HOLD: {spread_buy_hold:.3f}%"
            
            if len(sell_ret) > 0 and len(hold_ret) > 0:
                spread_sell_hold = (sell_ret[0] - hold_ret[0]) * 100
                result += f"\n  SELL vs HOLD: {spread_sell_hold:.3f}%"
        
        return result
    except Exception as e:
        return f"Error testing strategy: {str(e)}\nMake sure your SQL conditions are valid."

@tool("get_distribution_stats")
@rate_limit
def get_distribution_stats(date: str = None) -> str:
    """
    Get statistical distribution of firm characteristics at a specific date.
    
    Args:
        date: Date in format 'YYYY-MM-DD' (if None, uses latest date)
    
    Returns:
        Distribution statistics for all characteristics
    """
    try:
        conn = sqlite3.connect('firm_data.db')
        
        if date is None:
            cursor = conn.cursor()
            cursor.execute("SELECT MAX(datadate) FROM firms")
            date = cursor.fetchone()[0]
        
        query = f"""
        SELECT 
            COUNT(*) as n_firms,
            AVG(mve) as avg_mve, MIN(mve) as min_mve, MAX(mve) as max_mve,
            AVG(bm) as avg_bm, MIN(bm) as min_bm, MAX(bm) as max_bm,
            AVG(mom12m) as avg_mom12m, MIN(mom12m) as min_mom12m, MAX(mom12m) as max_mom12m
        FROM firms
        WHERE datadate = '{date}'
        """
        
        df = pd.read_sql_query(query, conn)
        conn.close()
        
        if df.empty:
            return f"No data found for date: {date}"
        
        result = f"Distribution statistics at {date}:\n\n"
        result += df.to_string(index=False)
        
        return result
    except Exception as e:
        return f"Error getting distribution: {str(e)}"

# ============================================
# Configure LLM
# ============================================
def get_rate_limited_llm():
    """Get LLM provider with rate limiting configured."""
    llm = AG.get_llm_provider()
    
    try:
        llm.max_retries = 5
        llm.timeout = 60
    except Exception as e:
        print(f"[Warning] Could not set LLM parameters: {e}")
    
    return llm

# ============================================
# Strategy Development Agent - WITH HISTORICAL LEARNING
# ============================================
strategy_agent = Agent(
    role="Empirical Quantitative Researcher",
    goal="Learn from 30 years of historical data (1990-2019) to develop systematic BUY/HOLD/SELL rules that predict future returns at December 2019",
    backstory="""
    You are an expert empirical quantitative researcher who discovers investment strategies through rigorous historical analysis.
    
    CRITICAL DATA UNDERSTANDING:
    - You have access to 30 YEARS of firm characteristics AND their subsequent returns (Jan 1990 - Nov 2019)
    - 'mve' = log(market value of equity), represents log firm size (STANDARDIZED z-score)
    - 'bm' = log(book to market) - value factor (STANDARDIZED z-score)
    - 'mom12m' = 12-month momentum (STANDARDIZED z-score)
    - 'ret_fwd_1' = forward return (the return AFTER characteristics were measured) - RAW return, NOT standardized
    - ALL characteristics are z-scores (mean=0, std=1)
    
    YOUR RESEARCH PROCESS:
    
    1. EXPLORE HISTORICAL PATTERNS (1990- November 2019):
       - Which characteristics predicted positive returns historically?
       - Do high momentum stocks outperform? Do value stocks outperform?
       - Are there combinations that work better (value + momentum)?
       - What z-score thresholds historically separated good from bad performers?
       - Has the relationship changed over time or been stable?
    
    2. EMPIRICAL ANALYSIS:
       - Analyze quintile returns: Do Q5 (high characteristic) stocks beat Q1 (low)?
       - Test ANY combination using flexible SQL conditions
       - Examples to explore:
         * "bm < -1.0 AND mom12m > 0.5" (value + positive momentum)
         * "(bm < -0.8 OR mom12m > 1.0) AND mve > 0" (value OR momentum, large caps only)
         * "bm < -1.2 AND mom12m > 0.7 AND mve > 0.3" (strong value + momentum + size)
         * "mom12m > 1.5 OR (bm < -1.5 AND mve > 0)" (strong momentum OR extreme value in large caps)
       - Look at spreads: How much better do good signals perform vs bad?
       - Test threshold sensitivity: Is -1.0 better than -1.2?
       - Consider statistical significance: Are patterns strong and consistent?
    
    3. DEVELOP EVIDENCE-BASED RULES:
       - Your rules must be JUSTIFIED by historical evidence
       - You have COMPLETE FLEXIBILITY in rule structure:
         * Simple: "bm < -1.2" (just value)
         * Compound AND: "bm < -1.2 AND mom12m > 0.7" (value AND momentum)
         * Compound OR: "bm < -1.5 OR mom12m > 1.5" (strong value OR strong momentum)
         * Complex: "(bm < -1.0 AND mom12m > 0.5) OR (mve > 1.0 AND mom12m > 1.2)" (value+momentum OR large+strong momentum)
         * Nested: "bm < -0.8 AND (mom12m > 0.7 OR mve > 1.0)" (value with momentum OR size)
       - Do NOT use arbitrary thresholds - let the data guide you
       - Use explore_threshold_sensitivity to find optimal cut-points
       - Consider: What thresholds maximize the spread between winners and losers?
    
    4. VALIDATE YOUR STRATEGY:
       - Test your proposed rules on historical data
       - Does your BUY signal historically produce positive returns?
       - Does your SELL signal historically produce negative returns?
       - Is the spread between BUY and SELL signals economically significant?
    
    5. SPECIFIC THRESHOLD SELECTION:
       - Be PRECISE in thresholds (not just round numbers like -1.0, use -1.17 if data supports it)
       - Consider market conditions at Dec 2019 
       - Balance between signal strength (fewer, stronger signals) and diversification (more signals)
       - Better to have too many signals than too few signals
    
    CRITICAL REQUIREMENTS:
    - Rules must be based on EMPIRICAL EVIDENCE from historical returns
    - Use tools to analyze characteristic-return relationships
    - Test your proposed strategy before finalizing
    - Provide evidence: "I choose this threshold because historically..."
    - Rules must be deterministic and implementable
    - Consider both individual characteristics AND combinations
    
    OUTPUT FORMAT:
    ===========================================
    EMPIRICAL TRADING STRATEGY
    ===========================================
    
    Historical Analysis Summary:
    - [What you learned from 30 years of data]
    - [Which characteristics predicted returns]
    - [Evidence for your threshold choices]
    
    BUY RULE:
    if [ANY complex z-score condition using AND/OR/NOT]:
        signal = BUY
    
    Examples of valid BUY rules:
    - "bm < -1.15 AND mom12m > 0.73" (simple AND)
    - "bm < -1.5 OR mom12m > 1.2" (simple OR)
    - "(bm < -1.0 AND mom12m > 0.5) OR mve > 1.5" (combination)
    - "bm < -0.8 AND (mom12m > 0.7 OR mve > 1.0)" (nested conditions)
    
    Historical Evidence for BUY:
    - [Average return of stocks meeting BUY condition: X%]
    - [Number of observations: N]
    - [Why these thresholds work]
    
    SELL RULE:
    if [exact z-score conditions]:
        signal = SELL
    
    Historical Evidence for SELL:
    - [Average return of stocks meeting SELL condition: Y%]
    - [Number of observations: N]
    - [Why these thresholds work]
    
    HOLD RULE:
    else:
        signal = HOLD
    
    Strategy Validation:
    - [BUY vs SELL spread: Z%]
    - [Expected signal distribution at Dec 2019]
    
    Rationale:
    - [Economic intuition supported by data]
    - [Why this strategy should work going forward]
    - [Market conditions at Dec 2019]
    
    ===========================================
    
    Be rigorous, empirical, and data-driven. Let the 30 years of historical evidence guide your rules.
    """,
    llm=get_rate_limited_llm(),
    memory=True,
    verbose=True,
    max_iter=20,  # More iterations for thorough analysis
    max_rpm=10
)

strategy_task = Task(
    description="""
    Learn from historical data (1990-2019) to develop systematic trading rules for December 2019.
    
    Available data:
    - 30 years of firm characteristics (mve, bm, mom12m) AND their forward returns
    - All data BEFORE Dec 2019 (training period)
    - Need to generate signals FOR Dec 2019 (out-of-sample)
    
    Your research process:
    1. Understand the data structure and available history
    2. Analyze which characteristics predicted returns historically
    3. Test different threshold combinations
    4. Develop final rules with empirical justification
    
    CRITICAL: 
    - Use historical returns to LEARN which characteristics work
    - Your rules must be justified by empirical evidence
    - Test your strategy before finalizing
    - Be precise with thresholds based on what data shows
    - Consider Dec 2019 market context
    
    Focus on:
    - Empirical evidence from 30 years of data
    - Characteristic-return relationships
    - Strategy backtesting and validation
    - Economically and statistically significant patterns
    
    Output: Evidence-based systematic rules for Dec 2019 trading.
    """,
    expected_output="""
    Complete empirical strategy with:
    1. Historical analysis summary (what predicts returns)
    2. Evidence-based BUY rule with specific thresholds
    3. Evidence-based SELL rule with specific thresholds
    4. HOLD rule (default)
    5. Rationale grounded in empirical evidence
    6. Expected performance at Dec 2019
    
    All rules must be justified by historical return analysis.
    """,
    agent=strategy_agent,
    tools=[
        get_database_schema,
        query_firm_database,
        get_distribution_stats,
        analyze_characteristic_returns,
        analyze_combined_characteristics,
        explore_threshold_sensitivity,
        test_strategy_historically
    ]
)

crew = Crew(
    agents=[strategy_agent],
    tasks=[strategy_task],
    process=Process.sequential,
    memory=False,
    verbose=True,
    max_rpm=10
)

# ============================================
# Main Execution
# ============================================
if __name__ == "__main__":

    selected_cols = variables  # Add your columns
    setup_firm_database('green_cleaned_pre_2020.csv', selected_columns=selected_cols)
    
    print("=" * 70)
    print("EMPIRICAL STRATEGY DEVELOPER")
    print("=" * 70)
    print("\nLearns from historical data (1990-2019) to develop trading rules")
    print("Uses 30 years of characteristic-return relationships")
    print("\nThe agent will:")
    print("  1. Analyze which characteristics predicted returns historically")
    print("  2. Test different threshold combinations")
    print("  3. Validate strategy with historical backtests")
    print("  4. Output evidence-based rules for December 2019")
    print("=" * 70)
    
    # Check if return column exists
    try:
        conn = sqlite3.connect('firm_data.db')
        cursor = conn.cursor()
        cursor.execute("PRAGMA table_info(firms)")
        columns = [col[1] for col in cursor.fetchall()]
        conn.close()
        
        if 'ret_fwd_1' not in columns:
            print("\n⚠ WARNING: 'ret_fwd_1' column not found in database!")
            print("The agent needs forward returns to learn from historical data.")
            print("Please ensure your CSV includes a 'ret_fwd_1' column with future returns.")
            response = input("\nContinue anyway? (yes/no): ")
            if response.lower() != 'yes':
                exit()
    except:
        pass
    
    input("\nPress Enter to start empirical strategy development...")
    
    try:
        print("\n🚀 Starting empirical analysis...\n")
        print("This may take longer as the agent analyzes 30 years of data...")
        
        # Record metadata
        import json
        metadata = {
            "timestamp": time.time(),
            "date": time.strftime("%Y-%m-%d %H:%M:%S"),
            "training_period": "1990-01-01 to 2019-11-30",
            "prediction_date": "2019-12-31",
            "method": "empirical_learning_from_historical_returns"
        }
        
        result = crew.kickoff()
        
        print("\n" + "=" * 70)
        print("EMPIRICAL STRATEGY OUTPUT")
        print("=" * 70)
        print(result)
        print("=" * 70)
        
        # Save with metadata
        with open('empirical_strategy.txt', 'w') as f:
            f.write("=" * 70 + "\n")
            f.write("EMPIRICAL STRATEGY - LEARNED FROM HISTORICAL DATA\n")
            f.write("=" * 70 + "\n")
            f.write(json.dumps(metadata, indent=2))
            f.write("\n\n" + "=" * 70 + "\n")
            f.write("STRATEGY OUTPUT\n")
            f.write("=" * 70 + "\n\n")
            f.write(str(result))
        
        print("\n✓ Strategy saved to 'empirical_strategy.txt'")
        print("\nThis strategy is based on 30 years of empirical evidence!")
        print("\nNext steps:")
        print("  1. Review the historical evidence and backtest results")
        print("  2. Implement the rules in Python")
        print("  3. Apply to December 2019 firms")
        print("  4. Track actual performance going forward")
        
    except KeyboardInterrupt:
        print("\n\n⏸ Interrupted.")
    except Exception as e:
        print(f"\n❌ Error: {e}")
        if "429" in str(e) or "RateLimitError" in str(e):
            print("\n🚨 Rate limit hit. Wait 60 seconds and try again.")

EMPIRICAL STRATEGY DEVELOPER

Learns from historical data (1990-2019) to develop trading rules
Uses 30 years of characteristic-return relationships

The agent will:
  1. Analyze which characteristics predicted returns historically
  2. Test different threshold combinations
  3. Validate strategy with historical backtests
  4. Output evidence-based rules for December 2019

⚠ WARNING: 'ret_fwd_1' column not found in database!
The agent needs forward returns to learn from historical data.
Please ensure your CSV includes a 'ret_fwd_1' column with future returns.


KeyboardInterrupt: Interrupted by user